# Revisiting the data filtering now that I have clearer set of test for what I need

In particular, there seem to be several examples where the data filtering that I set-up previously doesn't work

In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
%matplotlib inline
from IPython.core.pylabtools import figsize

import seaborn as sns
import plotly.express as px

import numpy as np
import pandas as pd
import polars as pl
import sqlite3

import statsmodels.formula.api as smf


## Simple checks

### Question 1: is the per share factored into the units (uom) column?

In [41]:
def get_raw_data(year, columns):
    """ Fetch and process all columns from a given year
     """
    #to do: filter only columns that use the correct unit of measure

    con = sqlite3.connect('data/processed/all10k.db')
    query = """ SELECT adsh, tag, ddate, dyear, version, coreg, qtrs, 
                 uom, value, footnote FROM num
                 WHERE (dyear=:year) AND {tagfilters}
            """
    tagfilters = "(" + \
                 " OR ".join([f"tag = :tag{i}" for i in range(len(columns))])\
                 + ")"
    query = query.format(tagfilters=tagfilters)

    params = {f'tag{i}': coli for i, coli in enumerate(columns)}
    params.update(year=str(year))

    data = pd.read_sql_query(query, con, params=params,
                             dtype={'value': float, 'adsh': str})
    return data

data = get_raw_data(2022, ['CommonStockDividendsPerShareDeclared', 'EarningsPerShareDiluted'])

In [42]:
data['uom'].unique()

array(['USD', 'ZAR', 'CAD'], dtype=object)

In [43]:
(data['uom'] == 'USD').sum()

8937

### answer:

The per-share units are __not__ taken into account in the uom column: we can filter on uom directly in the query

In [44]:
data[ data['uom'] != 'USD']

,adsh,tag,ddate,dyear,version,coreg,qtrs,uom,value,footnote
45,0001628280-22-017109,CommonStockDividendsPerShareDeclared,2022-03-31,2022,us-gaap/2021,None,1,ZAR,0.04,None
1178,0000049938-23-000015,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2022,None,4,CAD,1.46,None
1188,0000016875-23-000008,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2022,None,4,CAD,0.76,None
2254,0001062993-22-013882,EarningsPerShareDiluted,2022-02-28,2022,us-gaap/2021,None,4,CAD,-0.10,None
4740,0000895728-23-000008,EarningsPerShareDiluted,2022-03-31,2022,us-gaap/2022,None,1,CAD,0.95,None
4741,0000895728-23-000008,EarningsPerShareDiluted,2022-06-30,2022,us-gaap/2022,None,1,CAD,0.22,None
4742,0000895728-23-000008,EarningsPerShareDiluted,2022-09-30,2022,us-gaap/2022,None,1,CAD,0.63,None
4743,0000895728-23-000008,EarningsPerShareDiluted,2022-12-31,2022,us-gaap/2022,None,1,CAD,-0.53,None
4744,0000895728-23-000008,EarningsPerShareDiluted,2022-12-31,2022,us-gaap/2022,None,4,CAD,1.28,None
4995,0000016875-23-000008,EarningsPerShareDiluted,2022-12-31,2022,us-gaap/2022,None,4,CAD,3.77,None


In [45]:
from read_data import get_submissions
subs = get_submissions(2022)

In [46]:
subs.loc['0000950170-23-029420']

cik                                                       1737927
name                                           CANOPY GROWTH CORP
period_filed                                  2023-03-31 00:00:00
prevrpt                                                     False
url             https://www.sec.gov/ixviewer/ix.html?doc=/Arch...
Name: 0000950170-23-029420, dtype: object

### Are there any coregistrants?

In [54]:
data[~data['coreg'].isna()]

,adsh,tag,ddate,dyear,version,coreg,qtrs,uom,value,footnote
171,0001140361-22-045115,CommonStockDividendsPerShareDeclared,2022-08-31,2022,us-gaap/2022,RubiconTechnologyInc,0,USD,11.0000,None
248,0001407623-23-000042,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2021,RetailOpportunityInvestmentsPartnershipLP,4,USD,0.5600,None
685,0001032208-23-000008,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2022,SanDiegoGasAndElectricCompany,4,USD,0.8600,None
895,0001360604-23-000019,CommonStockDividendsPerShareDeclared,2022-07-31,2022,us-gaap/2022,HealthcareTrustOfAmericaInc,0,USD,4.8200,None
940,0000827052-23-000010,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2022,SouthernCaliforniaEdisonCompany,4,USD,3.0468,None
999,0000899715-23-000044,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2022,TangerFactoryOutletCentersInc.,4,USD,0.8025,None
1015,0000921825-23-000011,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2022,FirstIndustrialLP,4,USD,1.1800,None
1039,0000899689-23-000010,CommonStockDividendsPerShareDeclared,2022-12-31,2022,us-gaap/2022,VornadoRealtyLp,4,USD,2.1200,None
2991,0001558370-23-003042,EarningsPerShareDiluted,2022-12-31,2022,us-gaap/2021,LathamInvestmentHoldingsLp,4,USD,-0.0500,None
3137,0001407623-23-000042,EarningsPerShareDiluted,2022-12-31,2022,us-gaap/2021,RetailOpportunityInvestmentsPartnershipLP,4,USD,0.4200,None


In [56]:
data[data['qtrs'] != '4']

,adsh,tag,ddate,dyear,version,coreg,qtrs,uom,value,footnote
4,0001564590-22-007169,CommonStockDividendsPerShareDeclared,2022-01-31,2022,us-gaap/2021,None,0,USD,0.24,None
5,0001564590-22-011817,CommonStockDividendsPerShareDeclared,2022-03-31,2022,us-gaap/2021,None,0,USD,0.09,None
8,0001558370-22-002442,CommonStockDividendsPerShareDeclared,2022-01-31,2022,us-gaap/2021,None,0,USD,0.05,None
9,0001558370-22-002442,CommonStockDividendsPerShareDeclared,2022-01-31,2022,us-gaap/2021,None,3,USD,0.20,None
10,0001558370-22-002585,CommonStockDividendsPerShareDeclared,2022-02-28,2022,us-gaap/2021,None,0,USD,0.09,None
...,...,...,...,...,...,...,...,...,...,...
8929,0001901606-23-000093,EarningsPerShareDiluted,2022-09-30,2022,us-gaap/2023,None,2,USD,0.30,None
8932,0000032604-23-000044,EarningsPerShareDiluted,2022-03-31,2022,us-gaap/2023,None,1,USD,1.13,None
8933,0000032604-23-000044,EarningsPerShareDiluted,2022-06-30,2022,us-gaap/2023,None,1,USD,1.54,None
8934,0000032604-23-000044,EarningsPerShareDiluted,2022-09-30,2022,us-gaap/2023,None,1,USD,1.24,None


#### Additional filtering

It is worth looking into which of the coregistrants have a parent reported as well, and which entries for single quarters have a 4 quarter as well.

The main question here is what we want to filter out right away at the query stage.
- __uom:__ I think any reports that are not in dollars are not helpful to see in the table. These should be filtered.
- __qtrs:__ There is a case that companies that report only a couple of quarters shouldn't have their data filtered if these companies have only been listed for less than a year. It should, however, be noted as such.
- __coreg:__ I really can't think of a case where I would want to see the financials of child companies but not their parent companies... leaning toward filtering out all such data.

## Looking into special cases:

These cases all cover a specific problem: companies can report a fact multiple ways, or report sub-facts. For example, `MinorityInterest` and `RedeemableNoncontrollingInterestEquityCarryingAmount` could be reported alone, or, both reported because `RedeemableNoncontrolling...` is a sub item of `MinorityInterest`

1. Apple Inc
2. General Mills
3. EUROPEAN WAX CENTER, INC
4. Cheveron balance sheet - non-controlling interest carrying amount is double counted




In [8]:
from read_data import get_numbers, get_submissions

year = 2022

def balance_data():
    balance_cols = ['Assets', 'Liabilities', 'StockholdersEquity',
                    'AssetsCurrent', 'LiabilitiesCurrent',
                    'CurrentAssets/Liabilities', 'WorkingCapital/Debt',
                    ]
    
    validation_cols = [
                    'LiabilitiesAndStockholdersEquity',
                    'StockholdersEquityIncludingPortionAttributableToNoncontrollingInterest',
                    ]
    
    minority_equity_cols = [
                    'MinorityInterest',
                    'NonredeemableNoncontrollingInterest',
                    'RedeemableNoncontrollingInterestEquityCommonCarryingAmount',
                    'RedeemableNoncontrollingInterestEquityCarryingAmount',
                    'RedeemableNoncontrollingInterestEquityFairValue'
                    ]
    
    balance_cols += validation_cols
    balance_cols += minority_equity_cols
    
    data = get_numbers(year, balance_cols)
    
    # condense minority equity and set default to zero
    data['TotalMinorityEquity'] = data[minority_equity_cols].fillna(0.).sum(axis=1)
    data['TotalMinorityEquity_notes'] = pd.Series(dtype='string')

    return data

def income_data():
    cash_flow_cols = ['Revenues', 'CostOfRevenue', 'GrossProfit',
                      'OperatingExpenses', 'OperatingIncomeLoss',
                      'CommonStockDividendsPerShareDeclared',
                      'ProfitLoss', 'NetIncomeLoss'
                      ]

    validation_cols = ['PreferredStockDividendsIncomeStatementImpact',
                       'NetIncomeLossAvailableToCommonStockholdersBasic'
                       ]

    minority_income_cols = [
                         'NetIncomeLossAttributableToNoncontrollingInterest',
                         'NetIncomeLossAttributableToNoncontrollingInterestOfSubsidiary'
                        ]

    revenue_synonyms = ['Revenues', 'RevenueFromContractWithCustomerExcludingAssessedTax']
    cost_synonyms = ['CostOfRevenue', 'CostOfGoodsAndServicesSold']
    operating_cost_synonyms = ['OperatingExpenses', 'SellingGeneralAndAdministrativeExpense']

    cash_flow_cols += validation_cols
    cash_flow_cols += minority_income_cols
    cash_flow_cols += revenue_synonyms[1:]
    cash_flow_cols += cost_synonyms[1:]
    cash_flow_cols += operating_cost_synonyms[1:]

    data = get_numbers(year, cash_flow_cols)

    return data

In [9]:
subs = get_submissions(2022)

### Apple

In [10]:
data = income_data()

In [11]:
apple_data = data.loc[subs[subs['name'] == 'APPLE INC'].index[0]]

In [12]:
apple_data

CommonStockDividendsPerShareDeclared                                              0.9
CostOfGoodsAndServicesSold                                             223546000000.0
CostOfRevenue                                                                     NaN
GrossProfit                                                            170782000000.0
NetIncomeLoss                                                           99803000000.0
NetIncomeLossAttributableToNoncontrollingInterest                                 NaN
NetIncomeLossAttributableToNoncontrollingInterestOfSubsidiary                     NaN
NetIncomeLossAvailableToCommonStockholdersBasic                                   NaN
OperatingExpenses                                                       51345000000.0
OperatingIncomeLoss                                                    119437000000.0
PreferredStockDividendsIncomeStatementImpact                                      NaN
ProfitLoss                                            

The issue here is that we have _both_ `OperatingExpenses` and `SellingGeneralAndAdministrativeExpense` which is a sub-item of operating expenses.

In [13]:
data[data['Revenues'].isna()]

,CommonStockDividendsPerShareDeclared,CostOfGoodsAndServicesSold,CostOfRevenue,GrossProfit,NetIncomeLoss,NetIncomeLossAttributableToNoncontrollingInterest,NetIncomeLossAttributableToNoncontrollingInterestOfSubsidiary,NetIncomeLossAvailableToCommonStockholdersBasic,OperatingExpenses,OperatingIncomeLoss,...,NetIncomeLossAttributableToNoncontrollingInterest_notes,NetIncomeLossAttributableToNoncontrollingInterestOfSubsidiary_notes,NetIncomeLossAvailableToCommonStockholdersBasic_notes,OperatingExpenses_notes,OperatingIncomeLoss_notes,PreferredStockDividendsIncomeStatementImpact_notes,ProfitLoss_notes,RevenueFromContractWithCustomerExcludingAssessedTax_notes,Revenues_notes,SellingGeneralAndAdministrativeExpense_notes
adsh,,,,,,,,,,,,,,,,,,,,,
0000002488-23-000047,NaN,1.299800e+10,NaN,1.060300e+10,1.320000e+09,NaN,NaN,NaN,NaN,1.264000e+09,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
0000003545-23-000030,2.0,NaN,NaN,-1.476500e+07,1.245900e+07,-573000.0,NaN,NaN,NaN,-2.484400e+07,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
0000004127-22-000038,2.3,2.881200e+09,NaN,2.604300e+09,1.275200e+09,NaN,NaN,NaN,1.077300e+09,1.527000e+09,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
0000004127-23-000030,2.3,2.881200e+09,NaN,2.604300e+09,1.275200e+09,NaN,NaN,NaN,1.077300e+09,1.527000e+09,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
0000004281-23-000004,0.1,NaN,NaN,NaN,4.690000e+08,NaN,NaN,467000000.0,NaN,9.190000e+08,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0001929980-23-000010,NaN,NaN,NaN,NaN,-1.239600e+05,NaN,NaN,NaN,NaN,-1.373110e+05,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
0001935979-23-000009,NaN,NaN,NaN,NaN,-5.702790e+08,NaN,NaN,NaN,5.679320e+08,-5.679320e+08,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>
0001937653-23-000011,NaN,NaN,NaN,NaN,1.243410e+08,NaN,NaN,124341000.0,NaN,1.305280e+08,...,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>,<NA>


In [14]:
%run finance_logic.py

In [20]:
test = SubFields('Revenues', ['RevenueFromContractWithCustomerExcludingAssessedTax'])
d2 = test.impute_parent(data)
d2['Revenues'].isna().any()

False

Nice. Fixed for now.

### General Mills

This is an egregious example of the data being incorrectly fetched: the revenue is totally wrong compared to the statement.

In [31]:
raw_data = get_raw_data(2022, ['Revenues'])

In [32]:
adsh = subs[subs['name'] == 'GENERAL MILLS INC'].index[0]

In [33]:
raw_data[ raw_data['adsh'] == adsh]

,adsh,tag,ddate,dyear,version,coreg,qtrs,uom,value,footnote
82,0001193125-22-185257,Revenues,2022-05-31,2022,us-gaap/2021,None,4,USD,2.134300e+09,None


In [35]:
con = sqlite3.connect('data/processed/all10k.db')
query = """ SELECT * FROM num
             WHERE (dyear=:year) AND (adsh=:adsh);
        """

data = pd.read_sql_query(query, con, params={'year': 2022, 'adsh': '0001193125-22-185257'},
                         dtype={'value': float, 'adsh': str})

In [38]:
data[ data['tag'] == 'Revenues']

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote,dyear
280,0001193125-22-185257,Revenues,us-gaap/2021,None,2022-05-31,4,USD,2.134300e+09,None,2022


In [39]:
data[ data['tag'] == 'RevenueFromContractWithCustomerExcludingAssessedTax']

,adsh,tag,version,coreg,ddate,qtrs,uom,value,footnote,dyear
277,0001193125-22-185257,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2021,None,2022-05-31,4,USD,1.899280e+10,None,2022
278,0001193125-22-185257,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2021,None,2022-02-28,1,USD,4.537700e+09,None,2022
279,0001193125-22-185257,RevenueFromContractWithCustomerExcludingAssess...,us-gaap/2021,None,2022-05-31,1,USD,4.891200e+09,None,2022


Ok, this is extremely annoying! Revenues is reported for a subsidiary, while RevenueFrom.. is reported for the parent company. There is apparently no metadata that tells the two apart. I think this should be represented in the coreg field, but seems to have been left out!

I think the solution here is to take the subfield if it is larger than the parent.